# Dataset Preprocessing

## If unzip or bzip2 not installed

## Download and unzip UCI datasets

In [ ]:
%%bash
wget "http://archive.ics.uci.edu/ml/machine-learning-databases/00291/airfoil_self_noise.dat"
wget "http://archive.ics.uci.edu/ml/machine-learning-databases/00220/Relation%20Network%20(Directed).data" 
wget "https://archive.ics.uci.edu/ml/machine-learning-databases/cpu-performance/machine.data" 
wget "http://archive.ics.uci.edu/ml/machine-learning-databases/concrete/slump/slump_test.data" 
wget "http://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.data" 
wget "http://archive.ics.uci.edu/ml/machine-learning-databases/00243/yacht_hydrodynamics.data" 
wget "http://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls" 
wget "http://archive.ics.uci.edu/ml/machine-learning-databases/00424/2014%20and%202015%20CSM%20dataset.xlsx" 
wget "http://archive.ics.uci.edu/ml/machine-learning-databases/00265/CASP.csv" 
wget "http://archive.ics.uci.edu/ml/machine-learning-databases/00368/Facebook_metrics.zip" 
unzip Facebook_metrics.zip 
rm Facebook_metrics.zip 
wget "http://archive.ics.uci.edu/ml/machine-learning-databases/00335/online_video_dataset.zip" 
unzip online_video_dataset.zip 
rm online_video_dataset.zip 
wget "http://archive.ics.uci.edu/ml/machine-learning-databases/00332/OnlineNewsPopularity.zip" 
unzip OnlineNewsPopularity.zip 
rm OnlineNewsPopularity.zip 
wget "http://archive.ics.uci.edu/ml/machine-learning-databases/00206/slice_localization_data.zip" 
unzip slice_localization_data.zip 
rm slice_localization_data.zip 
wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00310/UJIndoorLoc.zip" 
unzip UJIndoorLoc.zip 
rm UJIndoorLoc.zip 

## Download and unzip news20

In [ ]:
%%bash
wget https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass/news20.scale.bz2
wget https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass/news20.t.scale.bz2
bzip2 -dk news20.scale.bz2
bzip2 -dk news20.t.scale.bz2

## Import preprocessing libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import sklearn.datasets as ds
from torch import torch

## Preprocessing UCI

In [ ]:
mat = pd.read_csv( "slump_test.data", header = 0, sep =",").values[:,1:]
np.save("UCI_dataset_01", np.delete(mat, -3, axis=1)[:,:-1])
np.save("UCI_dataset_02", np.delete(np.delete(mat, -2, axis=1) , -2, axis=1))

mat = pd.read_csv( "machine.data", header = None, sep =",").values[:,2:]
mat[mat == "?"] =None
mat = mat.astype(float)
mat[mat == None] = np.nan
mat = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(mat)
np.save("UCI_dataset_03", mat)

df = pd.read_excel( "2014 and 2015 CSM dataset.xlsx", header = 0)
Ratings_cols = ['Budget', 'Screens',
       'Sequel', 'Sentiment', 'Views', 'Likes', 'Dislikes', 'Comments',
       'Aggregate Followers', 'Ratings']
mat = df[Ratings_cols].values
mat = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(mat)
np.save("UCI_dataset_04", mat)

np.save("UCI_dataset_05", pd.read_csv( "yacht_hydrodynamics.data", header = None, sep="\s+").values)

np.save("UCI_dataset_06",pd.read_excel( "Concrete_Data.xls", header = 0).values.astype(float))

np.save("UCI_dataset_07", pd.read_csv("airfoil_self_noise.dat", header = None, sep ="\t").values)

mat = pd.read_csv( "communities.data", header = None, sep =",").values[:,5:]
mat[mat == "?"] =None
mat = mat.astype(float)
mat[mat == None] = np.nan
mat = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(mat)
np.save("UCI_dataset_08", mat)

np.save("UCI_dataset_09", pd.read_csv( "CASP.csv", header = 0)[['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'RMSD']].values.astype(float))

np.save("UCI_dataset_10", pd.read_csv("Relation Network (Directed).data", header = None, sep =",").values[:,1:].astype(float))


cols = ['duration', 'width', 'height', 'bitrate', 'framerate',
       'i', 'p', 'b', 'frames', 'i_size', 'p_size', 'b_size', 'size',
        'o_bitrate', 'o_framerate', 'o_width', 'o_height', 'umem',
       'utime']

np.save("UCI_dataset_11", pd.read_csv("transcoding_mesurment.tsv" , header = 0, sep = "\t")[cols].values)

df = pd.read_csv("UJIndoorLoc/trainingData.csv" , header = 0, sep = ",").iloc[:,:522]
lat = [col for col in df.columns if col != 'LONGITUDE']
long = [col for col in df.columns if col != 'LATITUDE']
np.save("UCI_dataset_12", df[lat].values)
np.save("UCI_dataset_13", df[long].values)

np.save("UCI_dataset_14", pd.read_csv("slice_localization_data.csv" , header = 0, sep = ",").values[:,1:])

## Preprocessing news20

In [ ]:
dataset_name = "news20"
X_train, y_train = ds.load_svmlight_file(dataset_name+'.scale', multilabel = False)
X_test, y_test = ds.load_svmlight_file(dataset_name+'.t.scale', multilabel = False)
    
cut_feature = 35830
#arbitrarly take 35830 first components to speed up computation time
X_train, X_test = X_train[:, :cut_feature], X_test[:, :cut_feature]

XT = X_train.transpose()
XTX = XT.dot(X_train)
dense_XTX = XTX.todense()
tensor_XTX = torch.tensor(dense_XTX).float()

# quite long process on cpu... (about 30 minutes)
with torch.no_grad():
    E, U = torch.symeig(tensor_XTX, eigenvectors=True)    
E = E.detach().numpy()
U = U.detach().numpy()
    
# keep 80 of variance by selecting the biggest eigen values
pos = E[E > 0.]
repartition = pos[::-1].cumsum()/pos.sum()
cut_eigen = 0.8
E_cut = E[-(repartition< cut_eigen).astype(int).sum():]
U_cut = U[:,-(repartition< cut_eigen).astype(int).sum():]
np.save(dataset_name+"_E_"+str(cut_eigen)+"_cut.npy",E_cut)
np.save(dataset_name+"_U_"+str(cut_eigen)+"_cut.npy",U_cut)

# transform features in principal components
np.save(dataset_name+"_X_train",X_train.dot(U_cut))
np.save(dataset_name+"_X_test",X_test.dot(U_cut))

## Remove raw datasets

In [ ]:
%%bash
rm "airfoil_self_noise.dat"
rm "Relation Network (Directed).data"
rm "machine.data"
rm "slump_test.data"
rm "communities.data"
rm "yacht_hydrodynamics.data"
rm "Concrete_Data.xls"
rm "2014 and 2015 CSM dataset.xlsx"
rm "CASP.csv"
rm "dataset_Facebook.csv"
rm "Facebook_metrics.txt"
rm "transcoding_mesurment.tsv"
rm "youtube_videos.tsv"
rm "README.txt"
rm -r "OnlineNewsPopularity"
rm "slice_localization_data.csv"
rm -r "UJIndoorLoc"
rm "news20.scale"
rm "news20.t.scale"